In [4]:
import pandas as pd
from tqdm import tqdm
import json
import os
import umap
import numpy as np
import scipy.sparse as sp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score, confusion_matrix


import stellargraph as sg
from stellargraph.mapper import FullBatchNodeGenerator

from stellargraph.layer import GCN

import warnings
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import activations, initializers, constraints, regularizers
from tensorflow.keras.layers import Input, Layer, Lambda, Dropout, Reshape, Dense
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras import layers, optimizers, losses, metrics, Model
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Data

## Read in edges, features, and targets

In [1]:
data_df= pd.read_csv('MLRD_UserData_20220120.csv', encoding='cp949')
data_df

In [2]:
data_sort = data_df.sort_values(by='signup_time')        
data_sort1 = data_sort.reset_index()
data_sort2 = data_sort1.drop('index', axis=1)
data_sort2

In [3]:
data_sort2['signup_country_en'].value_counts()

In [4]:
data1 = data_sort2[['user_id', 'no_of_days_since_last_trade', 'total_no_of_trade_times', 'accumulative_trading_amount_in_usdt',
                'signup_channel_type', 'signup_device_type', 'signup_country_en', 'max_kyc_level', 'first_deposit_method']]
data1

In [5]:
# 세 개의 컬럼에 대해 결측치 행 모두 제거
data3 =data1.dropna(axis=0)
data3.info()

data4 = data3.reset_index()
data5 = data4.drop('index', axis=1)
data5

In [6]:
rfm_df2 = pd.DataFrame()
rfm_df2['CustomerID'] = data5['user_id']
rfm_df2['Recency'] = data5['no_of_days_since_last_trade']            # 최근 거래 후 몇일 지났는지
rfm_df2['Frequency'] = data5['total_no_of_trade_times']              # 거래 총 수
rfm_df2['Monetary'] = data5['accumulative_trading_amount_in_usdt']   # 총 거래 금액(USDT)
rfm_df2

In [7]:
rfm_df1 = rfm_df2.reset_index()
rfm_df = rfm_df1.drop('index', axis=1)
rfm_df

In [12]:
### Log 변환을 통해 데이터 변환
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples

# Recency, Frequecny, Monetary 컬럼에 np.log1p() 로 Log Transformation
rfm_df['Recency_log'] = np.log1p(rfm_df['Recency'])
rfm_df['Frequency_log'] = np.log1p(rfm_df['Frequency'])
rfm_df['Monetary_log'] = np.log1p(rfm_df['Monetary'])

# Log Transformation 데이터에 StandardScaler 적용
X_features = rfm_df[['Recency_log','Frequency_log','Monetary_log']].values
X_features_scaled = StandardScaler().fit_transform(X_features)

kmeans = KMeans(n_clusters=3, random_state=0)
labels = kmeans.fit_predict(X_features_scaled)
rfm_df['cluster_label'] = labels

print('실루엣 스코어는 : {0:.3f}'.format(silhouette_score(X_features_scaled,labels)))     # cluster=3에서의 실루엣 점수(로그 변환 전보다는 스코어가 떨어짐)

실루엣 스코어는 : 0.366


In [8]:
data5['cluster'] = rfm_df['cluster_label']
data5

In [9]:
data6 = data5[(data5['cluster']==0) | (data5['cluster']==2)]
data6

In [10]:
data6['cluster'] = data6['cluster'].replace(2,1)
data7 = data6.reset_index()
data8 = data7.drop('index', axis=1)
data8

In [16]:
data8['cluster'].value_counts()

1    20460
0    17525
Name: cluster, dtype: int64

In [17]:
# 최종 경로 유형에 따라 엣지 df만들기
arr_app = np.array(data8[data8['signup_channel_type']=='App paid apk'].index)
arr_org = np.array(data8[data8['signup_channel_type']=='Organic traffic'].index)
arr_ref = np.array(data8[data8['signup_channel_type']=='Referral'].index)
arr_seo = np.array(data8[data8['signup_channel_type']=='SEO'].index)
arr_aff = np.array(data8[data8['signup_channel_type']=='Affiliate'].index)
arr_cam = np.array(data8[data8['signup_channel_type']=='Campaign promotion'].index)
arr_oth = np.array(data8[data8['signup_channel_type']=='Others'].index)

In [18]:
from tqdm import tqdm 

edges_df = pd.DataFrame()
start=[]
end=[]
# arrs=[arr_app, arr_org, arr_ref, arr_seo, arr_aff, arr_cam, arr_oth]
arrs = [arr_cam]

for arr in tqdm(arrs):
    for i in range(len(arr)-1):
        for j in range(i+1, len(arr)):
            start.append(arr[i])
            end.append(arr[j])

edges_df['source'] = start
edges_df['target'] = end        

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 342.39it/s]


In [81]:
print(len(set(start)))
len(set(end))

116


116

In [19]:
edges_df

,source,target
0,32,407
1,32,512
2,32,722
3,32,731
4,32,736
...,...,...
6781,34530,36065
6782,34530,36366
6783,35079,36065
6784,35079,36366


In [20]:
features_df = pd.DataFrame()

f1 = pd.get_dummies(data8['signup_device_type'])
f2 = pd.get_dummies(data8['signup_country_en'])
f3 = pd.get_dummies(data8['max_kyc_level'])
f4 = pd.get_dummies(data8['first_deposit_method'])


i=0
fs=[f1,f2,f3,f4]
for f in tqdm(fs):
    for col in f.columns:
        features_df[i] = f[col]
        i+=1
    
print(features_df)

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 124.97it/s]

       0    1    2    3    4    5    6    7    8    9    ...  94   95   96   \
0        0    0    1    0    0    0    0    0    0    0  ...    0    1    0   
1        1    0    0    0    0    0    0    0    0    0  ...    0    0    1   
2        1    0    0    0    0    0    0    0    0    0  ...    0    1    0   
3        0    0    1    0    0    0    0    0    0    0  ...    0    0    0   
4        0    1    0    0    0    0    0    0    0    0  ...    0    1    0   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
37980    1    0    0    0    0    0    0    0    0    0  ...    0    0    0   
37981    1    0    0    0    0    0    0    0    0    0  ...    0    1    0   
37982    0    0    1    0    0    0    0    0    0    0  ...    0    1    0   
37983    0    0    1    0    0    0    0    0    0    0  ...    0    0    1   
37984    0    0    1    0    0    0    0    0    1    0  ...    0    0    0   

       97   98   99   100  101  102  103  
0       

# RFM TARGET

In [21]:
targets_df1 = pd.DataFrame()
targets_df1['id'] = data8['user_id']
targets_df1['target'] = data8['cluster']
targets_df2 = targets_df1.reset_index()
targets_df = targets_df2.drop('index', axis=1)
targets_df

,id,target
0,602567404,0
1,602656046,0
2,602572035,1
3,602707370,0
4,602674754,0
...,...,...
37980,602737479,0
37981,602828191,1
37982,602823532,0
37983,602639489,1


In [22]:
targets_df['target'].value_counts()

1    20460
0    17525
Name: target, dtype: int64

In [23]:
features_df

,0,1,2,3,4,5,6,7,8,9,...,94,95,96,97,98,99,100,101,102,103
0,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37980,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
37981,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
37982,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
37983,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0


In [24]:
edges_df

,source,target
0,32,407
1,32,512
2,32,722
3,32,731
4,32,736
...,...,...
6781,34530,36065
6782,34530,36366
6783,35079,36065
6784,35079,36366


In [25]:
targets_df

,id,target
0,602567404,0
1,602656046,0
2,602572035,1
3,602707370,0
4,602674754,0
...,...,...
37980,602737479,0
37981,602828191,1
37982,602823532,0
37983,602639489,1


## StellarGraph Data

StellarGraph는 자체적인 그래프 데이터 구조를 가지고 있다.

In [28]:
import gc
gc.collect()      

15

In [29]:
G = sg.StellarGraph(features_df, edges_df)

In [30]:
print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 37985, Edges: 6786

 Node types:
  default: [37985]
    Features: float32 vector, length 104
    Edge types: default-default->default

 Edge types:
    default-default->default: [6786]
        Weights: all 1 (default)
        Features: none


## Train/Test/Val split

참고로, GCN은 purely supervised models(e.g. Random Forest)에 비해 label이 훨씬 적게 필요한 'semi-supervised model'이다. 

In [31]:
targets_df

,id,target
0,602567404,0
1,602656046,0
2,602572035,1
3,602707370,0
4,602674754,0
...,...,...
37980,602737479,0
37981,602828191,1
37982,602823532,0
37983,602639489,1


In [32]:
# 200 training examples
train_pages1, test_pages1 = train_test_split(targets_df, train_size=200)
val_pages1, test_pages1 = train_test_split(test_pages1, train_size=200)

In [33]:
train_pages1.shape, val_pages1.shape, test_pages1.shape

((200, 2), (200, 2), (37585, 2))

In [34]:
test_pages1['target'].value_counts()

1    20238
0    17347
Name: target, dtype: int64

# Pre-processing

## Target pre-processing

In [35]:
target_encoding = LabelBinarizer()  # 2개의 클래스가 있을 때, 이에 대한 벡터를 반환함
train_targets1 = target_encoding.fit_transform(train_pages1['target'])
val_targets1 = target_encoding.transform(val_pages1['target'])
test_targets1 = target_encoding.transform(test_pages1['target'])

In [36]:
train_targets1[:3]

array([[1],
       [0],
       [0]])

출력 layer에서 softmax 기능을 사용하는 neural network에 제공할 수 있도록 원핫 인코딩 형식이 필요함

## Graph Data Pre-processing

GCN은 기능과 그래프 인접 정보를 결합할 수 있는 강력한 심층 신경망 아키텍처이다. 이것은 컨볼루션 필터 역할을 하는 정규화된 인접 행렬에 이전 계층 값을 곱함으로써 달성된다. 이 곱셈의 결과로 이웃 노드의 기능이 집계되고 평소처럼 역 전파를 사용하여 전체 임베딩을 학습할 수 있다.

# GCN Model

실험을 더 빠르고 덜 복잡하게 하기 위해 StellarGraph API를 완전히 사용해보자.
graph data의 모든 전처리는 사실 StellarGraph가 모두 하고 있다. BatchGenerator 개체를 초기화하고 사용하기만 하면 된다.

In [37]:
# Initialize the generator
generator = FullBatchNodeGenerator(G, method="gcn")

# Use the .flow method to prepare it for use with GCN
train_gen1 = generator.flow(train_pages1.index, train_targets1)
val_gen1 = generator.flow(val_pages1.index, val_targets1)
test_gen1 = generator.flow(test_pages1.index, test_targets1)

Using GCN (local pooling) filters...


GCN 모델을 구축하는 것 또한 StellarGraph를 이용하면 매우 쉽다.

In [38]:
# Build necessary layers
gcn = GCN(
    layer_sizes=[32,32], activations=["relu", "relu"], generator=generator, dropout=0.5
    )

# Access the input and output tensors
x_inp, x_out = gcn.in_out_tensors()

# Pass the output tensor through the dense layer with sigmoid
predictions = layers.Dense(units=train_targets1.shape[1], activation="sigmoid")(x_out)

In [39]:
model = Model(inputs=x_inp, outputs=predictions)
model.compile(
    optimizer = optimizers.Adam(lr=0.01),
    loss = losses.binary_crossentropy,
    metrics = ["acc"]
)

C:\Users\admin\anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


generator 객체가 처리하므로, 모델에 모든 inputs들을 제공하는 것에 대해 걱정할 필요가 없다.

In [40]:
history = model.fit(
    train_gen1,
    epochs = 200,
    validation_data = val_gen1,
    verbose = 1,
    shuffle=False,         # True: 그래프 전체를 섞는다는 의미
#     callbacks=[es_callback]
)

Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 0.7078 - acc: 0.5300 - val_loss: 0.6941 - val_acc: 0.4800
Epoch 2/200
1/1 [==============================] - 0s 98ms/step - loss: 0.7021 - acc: 0.5000 - val_loss: 0.6955 - val_acc: 0.4850
Epoch 3/200
1/1 [==============================] - 0s 101ms/step - loss: 0.6860 - acc: 0.5400 - val_loss: 0.6924 - val_acc: 0.5350
Epoch 4/200
1/1 [==============================] - 0s 106ms/step - loss: 0.6793 - acc: 0.5750 - val_loss: 0.6888 - val_acc: 0.5300
Epoch 5/200
1/1 [==============================] - 0s 101ms/step - loss: 0.6745 - acc: 0.5650 - val_loss: 0.6866 - val_acc: 0.5400
Epoch 6/200
1/1 [==============================] - 0s 90ms/step - loss: 0.6815 - acc: 0.5600 - val_loss: 0.6866 - val_acc: 0.5400
Epoch 7/200
1/1 [==============================] - 0s 76ms/step - loss: 0.6873 - acc: 0.5600 - val_loss: 0.6868 - val_acc: 0.5400
Epoch 8/200
1/1 [==============================] - 0s 75ms/step - loss: 0.6762 - acc: 0.5

StellarGraph API를 이용하니 훈련 시간이 훨씬 더 빠르다.

# Model Evaluation

In [41]:
def evaluate_preds(true, pred):
    auc = roc_auc_score(true, pred)
    pr = average_precision_score(true, pred)
    bin_pred = [1 if p > 0.5 else 0 for p in pred]   # 0.5
    f_score = f1_score(true, bin_pred)
    print('ROC AUC:', auc)
    print('PR AUC:', pr)
    print('F1 score:', f_score)
    print(confusion_matrix(true, bin_pred, normalize='true'))

    return auc, pr, f_score

In [42]:
new_preds1 = model.predict(test_gen1)
new_preds1

array([[[0.5799192 ],
        [0.57999974],
        [0.56275296],
        ...,
        [0.50113624],
        [0.6097721 ],
        [0.61397517]]], dtype=float32)

In [43]:
test_targets1

array([[0],
       [0],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [44]:
test_targets1.ravel()       # 위의 것을 평평하게 펴준다.

array([0, 0, 1, ..., 0, 0, 0])

In [45]:
new_preds1.ravel()

array([0.5799192 , 0.57999974, 0.56275296, ..., 0.50113624, 0.6097721 ,
       0.61397517], dtype=float32)

In [46]:
# train:val:test=200:200:37585 썻을 때의 결과
auc, pr, f_score = evaluate_preds(test_targets1.ravel(), new_preds1[0].ravel())

ROC AUC: 0.5696582661485982
PR AUC: 0.5891777502648926
F1 score: 0.6852955508077515
[[0.14469361 0.85530639]
 [0.09660045 0.90339955]]


# RF Baseline

In [47]:
from sklearn.ensemble import RandomForestClassifier

In [48]:
rf = RandomForestClassifier()

rf.fit(features_df.loc[train_pages1.index, :], train_targets1.ravel())   # 노드피쳐와 타겟을 훈련(trainset index맞게)

test_preds = rf.predict_proba(features_df.loc[test_pages1.index, :])[:,1]  # 예측(testset index맞게)
evaluate_preds(test_targets1.ravel(), test_preds)

ROC AUC: 0.5605157349510047
PR AUC: 0.585998833308655
F1 score: 0.6222583632626325
[[0.40860091 0.59139909]
 [0.31939915 0.68060085]]


(0.5605157349510047, 0.585998833308655, 0.6222583632626325)

# Adding More Data

In [64]:
# 30000 training examples
train_pages, test_pages = train_test_split(targets_df, train_size=30000)
val_pages, test_pages = train_test_split(test_pages, train_size=3000)

train_targets = target_encoding.fit_transform(train_pages['target'])
val_targets = target_encoding.transform(val_pages['target'])
test_targets = target_encoding.transform(test_pages['target'])

In [65]:
# Initialize the generator
generator = FullBatchNodeGenerator(G, method="gcn")

# Use the .flow method to prepare it for use with GCN
train_gen = generator.flow(train_pages.index, train_targets)
val_gen = generator.flow(val_pages.index, val_targets)
test_gen = generator.flow(test_pages.index, test_targets)

Using GCN (local pooling) filters...


In [66]:
# Build necessary layers
gcn = GCN(
    layer_sizes=[32,32], activations=["relu", "relu"], generator=generator, dropout=0.5
    )

# Access the input and output tensors
x_inp, x_out = gcn.in_out_tensors()

# Pass the output tensor through the dense layer with sigmoid
predictions = layers.Dense(units=train_targets.shape[1], activation="sigmoid")(x_out)

In [67]:
model = Model(inputs=x_inp, outputs=predictions)
model.compile(
    optimizer = optimizers.Adam(lr=0.01),
    loss = losses.binary_crossentropy,
    metrics = ["acc"]
)

C:\Users\admin\anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [68]:
history = model.fit(
    train_gen,
    epochs = 200,
    validation_data = val_gen,
    verbose = 2,
    shuffle=False,         # True: 그래프 전체를 섞는다는 의미
#     callbacks=[es_callback]
)

Epoch 1/200
1/1 - 1s - loss: 0.6949 - acc: 0.5164 - val_loss: 0.6851 - val_acc: 0.5287
Epoch 2/200
1/1 - 0s - loss: 0.6916 - acc: 0.5425 - val_loss: 0.6838 - val_acc: 0.5583
Epoch 3/200
1/1 - 0s - loss: 0.6876 - acc: 0.5453 - val_loss: 0.6839 - val_acc: 0.5663
Epoch 4/200
1/1 - 0s - loss: 0.6867 - acc: 0.5449 - val_loss: 0.6840 - val_acc: 0.5757
Epoch 5/200
1/1 - 0s - loss: 0.6852 - acc: 0.5497 - val_loss: 0.6833 - val_acc: 0.5710
Epoch 6/200
1/1 - 0s - loss: 0.6846 - acc: 0.5530 - val_loss: 0.6825 - val_acc: 0.5657
Epoch 7/200
1/1 - 0s - loss: 0.6839 - acc: 0.5556 - val_loss: 0.6818 - val_acc: 0.5647
Epoch 8/200
1/1 - 0s - loss: 0.6833 - acc: 0.5559 - val_loss: 0.6811 - val_acc: 0.5667
Epoch 9/200
1/1 - 0s - loss: 0.6827 - acc: 0.5589 - val_loss: 0.6804 - val_acc: 0.5680
Epoch 10/200
1/1 - 0s - loss: 0.6825 - acc: 0.5578 - val_loss: 0.6798 - val_acc: 0.5727
Epoch 11/200
1/1 - 0s - loss: 0.6820 - acc: 0.5612 - val_loss: 0.6794 - val_acc: 0.5857
Epoch 12/200
1/1 - 0s - loss: 0.6810 - ac

In [69]:
new_preds = model.predict(test_gen)

In [70]:
# train:val:test=30000:3000:4985 썻을 때의 결과
auc, pr, f_score = evaluate_preds(test_targets.ravel(), new_preds[0].ravel())

ROC AUC: 0.6134569335201855
PR AUC: 0.6269593592691354
F1 score: 0.6776123983427957
[[0.29302124 0.70697876]
 [0.17550411 0.82449589]]


성능을 높이기 위해 데이터를 조금 더 추가하거나 epoch를 더 늘릴 여지가 있다.